In [ ]:
!nvidia-smi

In [ ]:
# --- CLEAN START ---
# Remove everything that can conflict
# !pip uninstall -y unsloth unsloth-zoo transformers accelerate peft datasets tokenizers huggingface-hub bitsandbytes sentencepiece

# --- PYTORCH (CUDA 11.8) ---
# !pip install torch --index-url https://download.pytorch.org/whl/cu118

# --- INSTALL MODERN UNSLOTH (the correct one for Qwen2.5) ---
# !pip install "unsloth==2026.2.1"

# --- INSTALL THE HF STACK COMPATIBLE WITH UNSLOTH 2026.x ---
# !pip install "transformers>=4.57.0" "accelerate>=0.34.1" "peft>=0.18.0" datasets sentencepiece bitsandbytes

# 1. Install Unsloth with the latest Colab-specific optimization
!pip install --force-reinstall --no-cache-dir --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# 2. Install essential backends for LoRA and 4-bit quantization
!pip install --no-deps "xformers<0.0.30" "trl<0.13.0" peft accelerate bitsandbytes

# 3. Install data handling and high-speed download utilities
!pip install datasets sentencepiece hf_transfer huggingface_hub

# 4. Enable HF_TRANSFER for 10x faster model loading into Colab's local disk
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [ ]:
from google.colab import drive
import torch
import os
drive.mount('/content/drive')

with open('/content/drive/MyDrive/TMP/HF_TOKEN.txt') as f:
    token = f.read().strip()
    if token:
        os.environ["HF_TOKEN"] = token
# 2. Check if HF_TOKEN is set in the environment
test_token=os.getenv("HF_TOKEN")
if test_token:
    print("HF_TOKEN is set in the environment.")
else:
    print("HF_TOKEN is NOT set in the environment. Please check your token file and try again.")
    raise ValueError("HF_TOKEN not found in environment variables.")

# 2. Detect Hardware for optimization
major_v, _ = torch.cuda.get_device_capability()
IS_A100 = True if major_v >= 8 else False
print(f"🚀 Hardware: {'A100 (using BF16)' if IS_A100 else 'T4 (using FP16)'}")

In [ ]:
import os

paths = {
    "Script": "/content/drive/MyDrive/scripts/train_scout_unsloth.py",
    "Data": "/content/drive/MyDrive/data/train/detective_finetune.jsonl"
}

for name, path in paths.items():
    if os.path.exists(path):
        print(f"✅ {name} found: {path}")
    else:
        print(f"❌ {name} MISSING: {path}")
        raise FileNotFoundError(f"{name} not found at {path}. Please check the path and try again.")

In [ ]:
import time

script_path = "/content/drive/MyDrive/scripts/train_scout_unsloth.py"

print("🎬 Initializing fine-tuning session...")
start_time = time.time()

# Run the training script
!python {script_path}

end_time = time.time()
elapsed_mins = (end_time - start_time) / 60

print(f"\n" + "="*40)
print(f"🏁 TRAINING SESSION COMPLETE")
print(f"⏱️ Total Time: {elapsed_mins:.2f} minutes")
print(f"📂 Output Location: /content/drive/MyDrive/outputs/detective-qwen-sft")
print("="*40)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2026-02-16 18:36:52.814618: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771267013.006387    1263 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771267013.069093    1263 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771267013.515901    1263 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771267013.515939    1263 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:17

🦥 Unsloth Zoo will now patch everything to make training faster!
Loading model Qwen/Qwen2.5-7B-Instruct with Unsloth...
==((====))==  Unsloth 2026.2.1: Fast Qwen2 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
model.safetensors.index.json: 112kB [00:00, 194MB/s]
model-00001-of-00002.safetensors: 100% 4.99G/4.99G [00:37<00:00, 134MB/s]     
model-00002-of-00002.safetensors: 100% 2.16G/2.16G [00:24<00:00, 86.7MB/s]   
Loading checkpoint shards: 100% 2/2 [00:25<00:00, 12.88s/it]
generation_config.json: 100% 271/271 [00:00<00:00, 1.67MB/s]
tokenizer_config.json: 7.36kB [00:00, 30.0MB/s]
vocab.json: 2.78MB [00:00, 72.2MB/s]
merges.txt: 